In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# !unzip "/content/drive/MyDrive/Dataset.zip" -d "/content/drive/MyDrive/CrisisMMImages"

In [ ]:
import pandas as pd

import torch
import os

from PIL import Image

import json
import matplotlib.pyplot as plt

import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from time import sleep
import warnings
from transformers import AdamW

from transformers import CLIPProcessor, CLIPModel
import torch.nn.functional as F

In [ ]:
# read annotations file

data = pd.read_csv("/content/drive/MyDrive/Text Benchmarking/LTI_Dataset.csv")


data = data.iloc[2058:]
data = data.reset_index()

for i in range(data.shape[0]):
  if(data['intent'][i]==2):
    data['intent'][i]=0

In [ ]:
data['intent'].value_counts()

In [ ]:
import re
Tweet = []
for tweettext in data["text"]:
  text = re.sub(r"http\S+", "", tweettext)
  text = re.sub(r"@\S+","",text)
  text = re.sub(r"@\S+","",text)
  text = re.sub(r".com$","",text)
  text = re.sub(r"@","",text)

  emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
  text = emoji_pattern.sub(r'',text)
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  #print(text)
  Tweet.append(text)

In [ ]:
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
nltk.download('omw-1.4')
nltk.download('wordnet')
wn=nltk.corpus.wordnet
wn_lemmas = set(wn.all_lemma_names())
len(wn_lemmas)
words.update(['vladimir', 'putin', 'zelenskyy', 'zelensky', 'russia', 'ukraine', 'trump', 'biden', 'joe', 'US', 'usa', 'nukes', 'kyiv', 'kiev'])
words.update(wn_lemmas)

In [ ]:
data["tweet_cleaned"] = pd.Series(Tweet)

text_cleaned = data["tweet_cleaned"] 
texts_new = []

for sentence in text_cleaned:
    text_new = " ".join(w.lower() for w in nltk.wordpunct_tokenize(sentence) if w.lower() in words or not w.isalpha()) 
    text_new = text_new.encode('ascii',errors='ignore').decode('ascii')
    texts_new.append(text_new)

data["tweet_cleaned"] = pd.Series(texts_new)
data

In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig
# tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')
# model=BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

import gensim
from gensim.summarization import summarize
from nltk.tokenize import sent_tokenize

for index in data.index:
  if len(data['tweet_cleaned'][index]) > 77:
    text = data['tweet_cleaned'][index]
    
    try:
      text = summarize(text, word_count = 50)
    except:
      list1 = list(text)
      list1 = list1[:77]
      text = ''.join(list1)

    data['tweet_cleaned'][index] = text
       # inputs = tokenizer.batch_encode_plus([pandas_df['tweet_cleaned'][index]],return_tensors='pt')
    # summary_ids = model.generate(inputs['input_ids'], early_stopping=True)
    # bart_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    # pandas_df['tweet_cleaned'][index] = bart_summary

data = data.reset_index()
data

In [ ]:
ids = (data['filename'].values)

texts = (data['tweet_cleaned'].values)

labels = (data['intent'].values)

In [ ]:
# DATASET

class Dataset(torch.utils.data.Dataset):
    """
    This is our custom dataset class which will load the text and their corresponding labels into Pytorch tensors
    """
    def __init__(self, labels, text, ids):
        self.labels = labels
        self.texts = text
        self.ids = ids

    def __getitem__(self, idx):
        sample = {}
        id = self.ids[idx]

        text = self.texts[idx]

        label = self.labels[idx]

        image_name = str(id) 

        img_path = os.path.join('/content/drive/MyDrive/CrisisMMImages/Dataset', image_name)

        try:
            sample["label"] = int(label)
            sample["text"] = text
            sample["image"] = img_path
        except Exception as e:
            print(e)
        
        return sample
    
    def __len__(self):
        return len(self.labels)
        

In [ ]:
dataset = Dataset(labels, texts, ids)

In [ ]:
data

In [ ]:
# split into training, validation and testing

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [2265, 400])

train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [1865, 400])

In [ ]:
## Specify the Hyper parameters 

BATCH_SIZE = 4
NUM_LABELS = 2   # since we have two labels - secuure and insecure i.e 0 and 1
EPOCHS = 3
LEARNING_RATE = 1e-3

# Check GPU
device = 'cpu'
print(device)

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled   = True

In [ ]:

## We call the dataloader class
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=0,
    shuffle=True,
    drop_last=True
 )

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=0,
    shuffle=True,
    drop_last=True
 )

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers=0,
    shuffle=True,
    drop_last=True
 )

dataloaders = {'Train': train_loader, 'Test': test_loader, 'Val': val_loader}



In [ ]:
#attach a classification layer
class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()

        self.fc1 = nn.Linear(1280, 128)
        self.fc2 = nn.Linear(128, 2)

        self.dropout = nn.Dropout(0.25)

    def forward(self, input):

      linear_out = self.dropout(F.relu(self.fc1(input)))    

      final_out = self.fc2(linear_out)    

      return final_out

In [ ]:
clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")

model = Model().to(device)

In [ ]:
#optimizer
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE, eps=1e-8)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

#Loss function
criterion = nn.CrossEntropyLoss()

In [ ]:
#training

train_loss = []
train_acc = []

val_loss = []
val_acc = []

for epoch in range(0, EPOCHS):
  print('-'*50)
  print('Epoch {}/{}'.format(epoch+1, EPOCHS))
  
  for phase in ['Train', 'Val']:

    batch_loss = 0.0000   #live loss
    batch_acc = 0.0000   #live accuracy
    
    y_pred = []
    y_true = []

    if phase == 'Train':
        model.train()
    else:
        model.eval()
    
    with tqdm(dataloaders[phase], unit="batch", desc=phase) as tepoch:

      for idx, batch in enumerate(tepoch):
        labels = batch["label"].to(device)
        text = batch["text"]

        imgs = []

        img_paths = batch["image"]

        for path in img_paths:

          imgs.append((Image.open(path)))

        
        inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True)

        clip_output = clip(**inputs)

        img_embed = clip_output.vision_model_output.pooler_output

        text_embed = clip_output.text_model_output.pooler_output

        concat = torch.cat((text_embed, img_embed), 1)

        output = model(concat)

        loss = criterion(output, labels)

        if phase == 'Train':

            #zero gradients
            optimizer.zero_grad() 

            # Backward pass  (calculates the gradients)
            loss.backward()     

            optimizer.step()             # Updates the weights    

        batch_loss += loss.item()

        _, preds = output.data.max(1)

        y_pred.extend(preds.tolist())
        y_true.extend(labels.tolist())
        
        tepoch.set_postfix(loss = batch_loss/(idx+1))

        #accumulate loss and accuracy

      if phase == 'Train':
        train_loss.append(batch_loss/(idx+1))
        train_acc.append(batch_acc)

      else:
        val_loss.append(batch_loss/(idx+1))
        val_acc.append(batch_acc)



In [ ]:
y_pred = []
y_true = []

with tqdm(dataloaders['Test'], unit="batch", desc=phase) as tepoch:
  for idx, batch in enumerate(tepoch):

    labels = batch["label"].to(device)
    text = batch["text"]

    imgs = []

    img_paths = batch["image"]

    for path in img_paths:

      imgs.append((Image.open(path)))
    
    inputs = processor(text=text, images=imgs, return_tensors="pt", padding=True)

    clip_output = clip(**inputs)

    img_embed = clip_output.vision_model_output.pooler_output

    text_embed = clip_output.text_model_output.pooler_output

    concat = torch.cat((text_embed, img_embed), 1)

    output = model(concat)

    _, preds = output.data.max(1)

    y_pred.extend(preds.tolist())
    y_true.extend(labels.tolist())



In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score

In [ ]:
accuracy_score(y_true,y_pred)

In [ ]:
f1_score(y_true, y_pred, average='macro')

In [ ]:
precision_score(y_true, y_pred, average='macro')

In [ ]:
recall_score(y_true, y_pred, average='macro')

In [ ]:
from imblearn.metrics import macro_averaged_mean_absolute_error 
macro_averaged_mean_absolute_error(y_true, y_pred)